In [ ]:
## This file implements neural networks with/without dropout and regularizer for p002ypresabs_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [89]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [90]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p002ypresabs_qual.csv')
df.shape

(253, 2033)

In [91]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [92]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
      ..
248    0
249    0
250    0
251    0
252    0
Name: pheno, Length: 253, dtype: int64

In [93]:
df.head()

,id,TTTTTTTGACTAAAATTAATGAAAAGTGAAAATAGTATTGGAACTCAATATCTTTAATGATTTAATGAATAATTTTTATTGAAAGCGATAATTCGTATTA,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTGACTAAAATTAATGAAAAGTGAAAATAGTATTGGAACTCAATATCTTTAATGATTTAATGAATAATTTTTATTGAAAGCGATAATTCGTATTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,...,group_8177,group_8643,group_8644,group_8645,group_8646,group_8815,group_8892,group_9007,group_9104,pheno
0,107,0,1,1,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,109,0,1,1,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,120335,0,1,1,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,120337,0,1,1,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [94]:
df['pheno'].value_counts()

0    220
1     30
2      3
Name: pheno, dtype: int64

In [95]:
df_clean = df.drop(columns=['id'])

In [96]:
df_clean.shape

(253, 2032)

In [97]:
df_clean.head()

,TTTTTTTGACTAAAATTAATGAAAAGTGAAAATAGTATTGGAACTCAATATCTTTAATGATTTAATGAATAATTTTTATTGAAAGCGATAATTCGTATTA,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTGACTAAAATTAATGAAAAGTGAAAATAGTATTGGAACTCAATATCTTTAATGATTTAATGAATAATTTTTATTGAAAGCGATAATTCGTATTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,...,group_8177,group_8643,group_8644,group_8645,group_8646,group_8815,group_8892,group_9007,group_9104,pheno
0,0,1,1,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,0,1,1,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 2032) (253,)


In [99]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 220), (1, 220), (2, 220)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [100]:
############# Fully-Connected Neural Network ################

In [101]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [102]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [103]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [104]:
dat

,0,test
0,CFBRSa26,0
1,NRS109,2
2,NRS112,0
3,NRS216,1
4,NRS021,0
...,...,...
193,CFBREBSa133,0
194,NRS209,2
195,NRS109,2
196,NRS209,2


In [105]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [68]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [69]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [70]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 561us/step - loss: 0.9054 - accuracy: 0.6255 - val_loss: 0.5772 - val_accuracy: 0.7525
Epoch 2/100
462/462 [==============================] - 0s 244us/step - loss: 0.5107 - accuracy: 0.8052 - val_loss: 0.4735 - val_accuracy: 0.7929
Epoch 3/100
462/462 [==============================] - 0s 298us/step - loss: 0.4337 - accuracy: 0.8182 - val_loss: 0.4013 - val_accuracy: 0.8131
Epoch 4/100
462/462 [==============================] - 0s 214us/step - loss: 0.4144 - accuracy: 0.8268 - val_loss: 0.3411 - val_accuracy: 0.8333
Epoch 5/100
462/462 [==============================] - 0s 213us/step - loss: 0.3856 - accuracy: 0.8247 - val_loss: 0.5292 - val_accuracy: 0.7374
Epoch 6/100
462/462 [==============================] - 0s 304us/step - loss: 0.4188 - accuracy: 0.8074 - val_loss: 0.2980 - val_accuracy: 0.8788
Epoch 7/100
462/462 [==============================] - 0s 368us/step - loss: 0.3238 

Epoch 57/100
462/462 [==============================] - 0s 239us/step - loss: 0.0344 - accuracy: 0.9957 - val_loss: 0.0679 - val_accuracy: 0.9697
Epoch 58/100
462/462 [==============================] - 0s 401us/step - loss: 0.0337 - accuracy: 0.9913 - val_loss: 0.0513 - val_accuracy: 0.9899
Epoch 59/100
462/462 [==============================] - 0s 364us/step - loss: 0.0299 - accuracy: 0.9957 - val_loss: 0.0467 - val_accuracy: 0.9899
Epoch 60/100
462/462 [==============================] - 0s 264us/step - loss: 0.0374 - accuracy: 0.9935 - val_loss: 0.0533 - val_accuracy: 0.9949
Epoch 61/100
462/462 [==============================] - 0s 317us/step - loss: 0.0310 - accuracy: 0.9935 - val_loss: 0.0492 - val_accuracy: 0.9899
Epoch 62/100
462/462 [==============================] - 0s 227us/step - loss: 0.0299 - accuracy: 0.9957 - val_loss: 0.0549 - val_accuracy: 0.9798
Epoch 63/100
462/462 [==============================] - 0s 364us/step - loss: 0.0373 - accuracy: 0.9870 - val_loss: 0.0742 -

In [21]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

198/198 [==============================] - 0s 150us/step
over-sampling test accuracy: 97.98%


In [71]:
pred = model1_over.predict_classes(X_test_over)
pred

array([0, 2, 0, 1, 0, 0, 2, 2, 2, 0, 1, 0, 1, 1, 0, 1, 2, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 2, 2, 1, 0, 0, 2, 1, 2, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       2, 2, 2, 1, 2, 2, 2, 1, 0, 2, 0, 0, 0, 2, 0, 0, 1, 2, 2, 0, 0, 1,
       0, 1, 0, 2, 1, 0, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 0, 0, 2, 1, 1, 0,
       0, 2, 0, 0, 2, 0, 1, 2, 1, 1, 1, 2, 0, 2, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 0, 0,
       2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 0, 2, 1, 1, 2, 0, 0, 2, 2, 1, 2, 1,
       0, 2, 0, 2, 1, 0, 2, 1, 1, 2, 0, 1, 2, 2, 1, 0, 2, 2, 2, 1, 2, 2,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 2, 0, 2, 1, 2, 0, 2, 2, 2, 1])

In [72]:
dat['pred'] = pred
dat

,0,test,pred
0,CFBRSa26,0,0
1,NRS109,2,2
2,NRS112,0,0
3,NRS216,1,1
4,NRS021,0,0
...,...,...,...
193,CFBREBSa133,0,0
194,NRS209,2,2
195,NRS109,2,2
196,NRS209,2,2


In [73]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [74]:
dat_proba1

,0,1,2
0,0.999852,1.476760e-04,5.399214e-12
1,0.000256,3.009203e-03,9.967352e-01
2,0.998466,1.533670e-03,9.995098e-10
3,0.007553,9.924409e-01,5.932516e-06
4,0.999889,1.113699e-04,2.815641e-11
...,...,...,...
193,0.999995,5.957744e-07,4.386633e-06
194,0.000005,3.933857e-05,9.999559e-01
195,0.000256,3.009205e-03,9.967352e-01
196,0.000005,3.933857e-05,9.999559e-01


In [75]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [76]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p002yp.csv", index = False,
         header=None)

In [24]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 254us/step - loss: 0.0148 - accuracy: 0.9957 - val_loss: 0.0327 - val_accuracy: 0.9848
Epoch 2/100
462/462 [==============================] - 0s 210us/step - loss: 0.0105 - accuracy: 0.9978 - val_loss: 0.0364 - val_accuracy: 0.9899
Epoch 3/100
462/462 [==============================] - 0s 206us/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.0414 - val_accuracy: 0.9848
Epoch 4/100
462/462 [==============================] - 0s 208us/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 0.0379 - val_accuracy: 0.9899
Epoch 5/100
462/462 [==============================] - 0s 212us/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.0334 - val_accuracy: 0.9899
Epoch 6/100
462/462 [==============================] - 0s 258us/step - loss: 0.0077 - accuracy: 1.0000 - val_loss: 0.0537 - val_accuracy: 0.9798
Epoch 7/100
462/462 [==============================] - 0s 272us/step - loss: 0.0066 

Epoch 57/100
462/462 [==============================] - 0s 207us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0394 - val_accuracy: 0.9848
Epoch 58/100
462/462 [==============================] - 0s 233us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0426 - val_accuracy: 0.9848
Epoch 59/100
462/462 [==============================] - 0s 322us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0309 - val_accuracy: 0.9899
Epoch 60/100
462/462 [==============================] - 0s 257us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0611 - val_accuracy: 0.9798
Epoch 61/100
462/462 [==============================] - 0s 215us/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0326 - val_accuracy: 0.9899
Epoch 62/100
462/462 [==============================] - 0s 188us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0417 - val_accuracy: 0.9899
Epoch 63/100
462/462 [==============================] - 0s 206us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0473 -

In [25]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 99.99%


In [106]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [107]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [108]:
y_prob = df_proba[df_proba['phage']=='p002ypresabs_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[9.99852300e-01, 1.47676020e-04, 5.39921360e-12],
       [2.55562340e-04, 3.00920340e-03, 9.96735160e-01],
       [9.98466400e-01, 1.53366980e-03, 9.99509800e-10],
       [7.55309500e-03, 9.92440940e-01, 5.93251570e-06],
       [9.99888660e-01, 1.11369925e-04, 2.81564110e-11],
       [9.99773800e-01, 2.26175000e-04, 7.29222850e-12],
       [3.02992940e-08, 2.11305330e-03, 9.97886960e-01],
       [3.02992940e-08, 2.11305330e-03, 9.97886960e-01],
       [2.55562340e-04, 3.00920340e-03, 9.96735160e-01],
       [1.00000000e+00, 2.96427060e-09, 5.35099860e-17],
       [3.38997300e-03, 9.96609450e-01, 5.71683800e-07],
       [1.00000000e+00, 2.86100080e-09, 1.50830130e-15],
       [5.45560430e-03, 9.94544400e-01, 2.63903830e-08],
       [8.71244400e-03, 9.88195060e-01, 3.09258330e-03],
       [9.99698640e-01, 3.01340920e-04, 5.24773700e-11],
       [3.69098700e-03, 9.96309000e-01, 2.36460100e-08],
       [3.02992940e-08, 2.11305330e-03, 9.97886960e-01],
       [4.69896070e-04, 9.99529

In [109]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [110]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.9980104071013162

In [111]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [112]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.9980104071013162

In [113]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [114]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [115]:
dat2

,0,test
0,NRS109,2
1,NRS109,2
2,NRS222,0
3,NRS109,2
4,GA50245,0
...,...,...
193,NRS148,2
194,NRS266,1
195,NRS109,2
196,NRS149,0


In [116]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [29]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [82]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [83]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 425us/step - loss: 0.6430 - accuracy: 0.7078 - val_loss: 0.5176 - val_accuracy: 0.7677
Epoch 2/100
462/462 [==============================] - 0s 204us/step - loss: 0.4556 - accuracy: 0.8052 - val_loss: 0.4375 - val_accuracy: 0.8283
Epoch 3/100
462/462 [==============================] - 0s 247us/step - loss: 0.3673 - accuracy: 0.8398 - val_loss: 0.3551 - val_accuracy: 0.8384
Epoch 4/100
462/462 [==============================] - 0s 271us/step - loss: 0.3258 - accuracy: 0.8485 - val_loss: 0.3388 - val_accuracy: 0.8737
Epoch 5/100
462/462 [==============================] - 0s 267us/step - loss: 0.2725 - accuracy: 0.9004 - val_loss: 0.3140 - val_accuracy: 0.8535
Epoch 6/100
462/462 [==============================] - 0s 201us/step - loss: 0.2586 - accuracy: 0.8939 - val_loss: 0.2765 - val_accuracy: 0.8939
Epoch 7/100
462/462 [==============================] - 0s 202us/step - loss: 0.2216 

Epoch 57/100
462/462 [==============================] - 0s 180us/step - loss: 0.0337 - accuracy: 0.9892 - val_loss: 0.0814 - val_accuracy: 0.9646
Epoch 58/100
462/462 [==============================] - 0s 232us/step - loss: 0.0219 - accuracy: 0.9935 - val_loss: 0.0578 - val_accuracy: 0.9697
Epoch 59/100
462/462 [==============================] - 0s 240us/step - loss: 0.0187 - accuracy: 0.9978 - val_loss: 0.0946 - val_accuracy: 0.9646
Epoch 60/100
462/462 [==============================] - 0s 220us/step - loss: 0.0206 - accuracy: 0.9978 - val_loss: 0.0801 - val_accuracy: 0.9646
Epoch 61/100
462/462 [==============================] - 0s 178us/step - loss: 0.0202 - accuracy: 0.9935 - val_loss: 0.0612 - val_accuracy: 0.9798
Epoch 62/100
462/462 [==============================] - 0s 188us/step - loss: 0.0164 - accuracy: 1.0000 - val_loss: 0.0880 - val_accuracy: 0.9646
Epoch 63/100
462/462 [==============================] - 0s 175us/step - loss: 0.0171 - accuracy: 0.9978 - val_loss: 0.0836 -

In [33]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

198/198 [==============================] - 0s 178us/step
over-sampling test accuracy: 96.97%


In [84]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([2, 2, 0, 2, 0, 2, 0, 1, 0, 2, 0, 1, 0, 1, 1, 2, 1, 2, 1, 2, 0, 1,
       2, 1, 1, 2, 1, 2, 1, 0, 2, 0, 1, 1, 1, 1, 0, 1, 2, 1, 0, 0, 2, 1,
       2, 0, 0, 2, 1, 1, 1, 2, 0, 1, 1, 2, 2, 0, 2, 0, 0, 1, 1, 2, 0, 2,
       1, 1, 0, 0, 1, 1, 2, 1, 2, 2, 0, 0, 0, 0, 1, 1, 0, 1, 2, 0, 1, 0,
       2, 0, 2, 1, 1, 1, 2, 2, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 2, 0, 2, 1,
       0, 2, 1, 1, 1, 1, 0, 0, 0, 2, 0, 2, 1, 2, 1, 2, 2, 2, 1, 0, 1, 1,
       1, 1, 2, 2, 0, 2, 0, 2, 2, 0, 1, 2, 0, 0, 1, 2, 2, 2, 0, 2, 1, 0,
       0, 2, 2, 2, 1, 1, 0, 2, 1, 2, 0, 0, 2, 0, 2, 2, 1, 1, 1, 2, 1, 2,
       0, 1, 0, 1, 2, 1, 2, 1, 0, 0, 0, 0, 1, 2, 2, 1, 1, 2, 1, 2, 0, 2])

In [85]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,NRS109,2,2
1,NRS109,2,2
2,NRS222,0,0
3,NRS109,2,2
4,GA50245,0,0
...,...,...,...
193,NRS148,2,2
194,NRS266,1,1
195,NRS109,2,2
196,NRS149,0,0


In [86]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [87]:
dat_proba2

,0,1,2
0,2.544798e-04,0.000465,9.992802e-01
1,2.544798e-04,0.000465,9.992802e-01
2,9.981701e-01,0.001830,9.083791e-10
3,2.544798e-04,0.000465,9.992802e-01
4,9.991456e-01,0.000854,6.293267e-09
...,...,...,...
193,8.168285e-08,0.000727,9.992730e-01
194,2.343134e-04,0.999761,4.523713e-06
195,2.544798e-04,0.000465,9.992802e-01
196,5.229113e-01,0.477087,1.622369e-06


In [88]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [89]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p002yp.csv", index = False,
         header=None)

In [37]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 263us/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 0.0699 - val_accuracy: 0.9697
Epoch 2/100
462/462 [==============================] - 0s 255us/step - loss: 0.0113 - accuracy: 1.0000 - val_loss: 0.0959 - val_accuracy: 0.9747
Epoch 3/100
462/462 [==============================] - 0s 235us/step - loss: 0.0099 - accuracy: 0.9978 - val_loss: 0.0660 - val_accuracy: 0.9747
Epoch 4/100
462/462 [==============================] - 0s 239us/step - loss: 0.0073 - accuracy: 1.0000 - val_loss: 0.0906 - val_accuracy: 0.9697
Epoch 5/100
462/462 [==============================] - 0s 269us/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.1330 - val_accuracy: 0.9596
Epoch 6/100
462/462 [==============================] - 0s 214us/step - loss: 0.0071 - accuracy: 1.0000 - val_loss: 0.1111 - val_accuracy: 0.9646
Epoch 7/100
462/462 [==============================] - 0s 205us/step - loss: 0.0057 

Epoch 57/100
462/462 [==============================] - 0s 187us/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.0667 - val_accuracy: 0.9798
Epoch 58/100
462/462 [==============================] - 0s 181us/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0800 - val_accuracy: 0.9747
Epoch 59/100
462/462 [==============================] - 0s 183us/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0806 - val_accuracy: 0.9747
Epoch 60/100
462/462 [==============================] - 0s 180us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0830 - val_accuracy: 0.9747
Epoch 61/100
462/462 [==============================] - 0s 180us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0847 - val_accuracy: 0.9697
Epoch 62/100
462/462 [==============================] - 0s 182us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0847 - val_accuracy: 0.9747
Epoch 63/100
462/462 [==============================] - 0s 183us/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0777 -

In [38]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 100.00%


In [117]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [118]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [119]:
y_prob2 = df_proba2[df_proba2['phage']=='p002ypresabs_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[2.54479800e-04, 4.65349440e-04, 9.99280150e-01],
       [2.54479800e-04, 4.65349440e-04, 9.99280150e-01],
       [9.98170140e-01, 1.82982660e-03, 9.08379100e-10],
       [2.54479800e-04, 4.65349440e-04, 9.99280150e-01],
       [9.99145600e-01, 8.54330400e-04, 6.29326700e-09],
       [2.54479800e-04, 4.65349440e-04, 9.99280150e-01],
       [9.96754940e-01, 3.24509390e-03, 3.90328230e-08],
       [2.86995750e-04, 9.99712900e-01, 8.05633600e-08],
       [1.00000000e+00, 4.73811100e-09, 1.06927760e-13],
       [2.54479800e-04, 4.65349440e-04, 9.99280150e-01],
       [9.99763200e-01, 2.36815600e-04, 2.30747400e-10],
       [2.82478230e-04, 9.99717060e-01, 4.84786000e-07],
       [9.98798850e-01, 1.20102940e-03, 6.36851000e-08],
       [2.84632390e-05, 9.94989450e-01, 4.98202600e-03],
       [2.11131560e-04, 9.99770000e-01, 1.88034560e-05],
       [8.16828500e-08, 7.26874200e-04, 9.99273000e-01],
       [5.20176600e-04, 9.99479500e-01, 3.98754280e-07],
       [1.26163780e-07, 3.97753

In [120]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.9972834404652587

In [121]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.9972834404652587

In [122]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [123]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [124]:
dat3

,0,test
0,CFBREBSa110,0
1,CFBREBSa131,1
2,NRS148,2
3,NRS169,1
4,NRS073,0
...,...,...
193,NRS001,1
194,NRS191,0
195,NRS207,0
196,CA26,0


In [125]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [94]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [95]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [96]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 422us/step - loss: 0.7285 - accuracy: 0.6840 - val_loss: 0.5866 - val_accuracy: 0.7778
Epoch 2/100
462/462 [==============================] - 0s 226us/step - loss: 0.4820 - accuracy: 0.8009 - val_loss: 0.4873 - val_accuracy: 0.7929
Epoch 3/100
462/462 [==============================] - 0s 235us/step - loss: 0.4086 - accuracy: 0.8160 - val_loss: 0.4218 - val_accuracy: 0.8182
Epoch 4/100
462/462 [==============================] - 0s 309us/step - loss: 0.3403 - accuracy: 0.8723 - val_loss: 0.3843 - val_accuracy: 0.8434
Epoch 5/100
462/462 [==============================] - 0s 300us/step - loss: 0.3200 - accuracy: 0.8701 - val_loss: 0.3812 - val_accuracy: 0.8131
Epoch 6/100
462/462 [==============================] - 0s 301us/step - loss: 0.3418 - accuracy: 0.8636 - val_loss: 0.3595 - val_accuracy: 0.8232
Epoch 7/100
462/462 [==============================] - 0s 285us/step - loss: 0.2584 

Epoch 57/100
462/462 [==============================] - 0s 212us/step - loss: 0.0266 - accuracy: 0.9978 - val_loss: 0.1498 - val_accuracy: 0.9545
Epoch 58/100
462/462 [==============================] - 0s 212us/step - loss: 0.0228 - accuracy: 0.9978 - val_loss: 0.0905 - val_accuracy: 0.9747
Epoch 59/100
462/462 [==============================] - 0s 349us/step - loss: 0.0184 - accuracy: 0.9978 - val_loss: 0.0995 - val_accuracy: 0.9697
Epoch 60/100
462/462 [==============================] - 0s 307us/step - loss: 0.0181 - accuracy: 0.9978 - val_loss: 0.0896 - val_accuracy: 0.9747
Epoch 61/100
462/462 [==============================] - 0s 180us/step - loss: 0.0226 - accuracy: 1.0000 - val_loss: 0.0894 - val_accuracy: 0.9747
Epoch 62/100
462/462 [==============================] - 0s 179us/step - loss: 0.0192 - accuracy: 0.9957 - val_loss: 0.0865 - val_accuracy: 0.9697
Epoch 63/100
462/462 [==============================] - 0s 284us/step - loss: 0.0180 - accuracy: 1.0000 - val_loss: 0.0901 -

In [46]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

198/198 [==============================] - 0s 198us/step
over-sampling test accuracy: 97.98%


In [97]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([0, 1, 2, 1, 0, 1, 1, 2, 1, 0, 0, 2, 1, 1, 0, 2, 0, 2, 0, 2, 0, 2,
       1, 0, 1, 2, 2, 2, 0, 2, 2, 1, 1, 0, 1, 2, 1, 0, 0, 2, 2, 1, 0, 0,
       1, 1, 0, 1, 2, 1, 2, 1, 0, 0, 1, 2, 2, 1, 1, 2, 1, 0, 2, 2, 2, 1,
       0, 1, 1, 2, 1, 1, 0, 1, 1, 2, 2, 1, 1, 2, 2, 0, 2, 0, 0, 1, 0, 2,
       2, 1, 2, 1, 2, 0, 1, 0, 0, 1, 1, 0, 0, 2, 1, 1, 0, 2, 0, 2, 0, 1,
       2, 2, 0, 1, 2, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 2, 2, 0,
       1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 0, 0, 2, 1, 1, 1, 2,
       1, 0, 0, 2, 2, 0, 1, 2, 2, 2, 2, 1, 1, 0, 2, 0, 1, 2, 0, 0, 0, 1,
       1, 1, 2, 0, 2, 2, 2, 0, 0, 1, 2, 0, 1, 2, 0, 2, 1, 1, 0, 0, 0, 0])

In [98]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,CFBREBSa110,0,0
1,CFBREBSa131,1,1
2,NRS148,2,2
3,NRS169,1,1
4,NRS073,0,0
...,...,...,...
193,NRS001,1,1
194,NRS191,0,0
195,NRS207,0,0
196,CA26,0,0


In [99]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [100]:
dat_proba3

,0,1,2
0,9.995762e-01,0.000424,1.239028e-10
1,2.607985e-02,0.973920,4.181065e-10
2,3.483531e-10,0.000689,9.993107e-01
3,2.207318e-03,0.995000,2.792645e-03
4,9.998268e-01,0.000173,1.205728e-14
...,...,...,...
193,3.822547e-03,0.996177,1.056541e-07
194,9.188336e-01,0.081166,3.952307e-08
195,9.996057e-01,0.000394,1.105371e-10
196,9.999986e-01,0.000001,9.736989e-13


In [101]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [102]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p002yp.csv", index = False,
         header=None)

In [50]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 257us/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.0843 - val_accuracy: 0.9848
Epoch 2/100
462/462 [==============================] - 0s 215us/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.0760 - val_accuracy: 0.9848
Epoch 3/100
462/462 [==============================] - 0s 198us/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.0696 - val_accuracy: 0.9848
Epoch 4/100
462/462 [==============================] - 0s 211us/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.0897 - val_accuracy: 0.9848
Epoch 5/100
462/462 [==============================] - 0s 194us/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.0713 - val_accuracy: 0.9848
Epoch 6/100
462/462 [==============================] - 0s 220us/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 0.0724 - val_accuracy: 0.9848
Epoch 7/100
462/462 [==============================] - 0s 264us/step - loss: 0.0048 

Epoch 57/100
462/462 [==============================] - 0s 176us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0886 - val_accuracy: 0.9848
Epoch 58/100
462/462 [==============================] - 0s 212us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0732 - val_accuracy: 0.9848
Epoch 59/100
462/462 [==============================] - 0s 294us/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.1034 - val_accuracy: 0.9747
Epoch 60/100
462/462 [==============================] - 0s 356us/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0802 - val_accuracy: 0.9848
Epoch 61/100
462/462 [==============================] - 0s 219us/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0989 - val_accuracy: 0.9747
Epoch 62/100
462/462 [==============================] - 0s 181us/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0951 - val_accuracy: 0.9848
Epoch 63/100
462/462 [==============================] - 0s 371us/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.0782 -

In [51]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 100.00%


In [126]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [127]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [128]:
y_prob3 = df_proba3[df_proba3['phage']=='p002ypresabs_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[9.99576150e-01, 4.23892840e-04, 1.23902780e-10],
       [2.60798520e-02, 9.73920170e-01, 4.18106470e-10],
       [3.48353120e-10, 6.89307050e-04, 9.99310730e-01],
       [2.20731760e-03, 9.95000100e-01, 2.79264450e-03],
       [9.99826850e-01, 1.73138690e-04, 1.20572760e-14],
       [1.33726550e-05, 9.99986650e-01, 1.79746240e-08],
       [2.89208330e-06, 9.93311400e-01, 6.68580830e-03],
       [3.48353120e-10, 6.89307050e-04, 9.99310730e-01],
       [2.24777940e-03, 9.97752000e-01, 1.84453800e-07],
       [5.43817340e-01, 4.56182630e-01, 1.28422650e-08],
       [9.99985200e-01, 1.48233200e-05, 1.13038530e-16],
       [2.84649430e-04, 3.68349300e-04, 9.99347030e-01],
       [1.76153390e-02, 9.82384400e-01, 2.00859550e-07],
       [3.82255060e-03, 9.96177300e-01, 1.05654110e-07],
       [9.99992000e-01, 7.95952900e-06, 4.14285670e-09],
       [2.84649430e-04, 3.68349300e-04, 9.99347030e-01],
       [9.99087200e-01, 9.12809450e-04, 3.87899700e-10],
       [2.84649430e-04, 3.68349

In [129]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.9934955616773798

In [130]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.9934955616773798

In [131]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [132]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [133]:
dat4

,0,test
0,NRS110,1
1,NRS216,1
2,NRS386,1
3,CFBRSa25,0
4,BCH-SA-03,1
...,...,...
193,NRS216,1
194,CFBREBSa110,0
195,NRS148,2
196,GA27,0


In [134]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [107]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [108]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [109]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 538us/step - loss: 0.7588 - accuracy: 0.6039 - val_loss: 0.5309 - val_accuracy: 0.7273
Epoch 2/100
462/462 [==============================] - 0s 272us/step - loss: 0.5450 - accuracy: 0.7727 - val_loss: 0.4666 - val_accuracy: 0.7879
Epoch 3/100
462/462 [==============================] - 0s 218us/step - loss: 0.4562 - accuracy: 0.8052 - val_loss: 0.4063 - val_accuracy: 0.8081
Epoch 4/100
462/462 [==============================] - 0s 342us/step - loss: 0.3847 - accuracy: 0.8095 - val_loss: 0.3772 - val_accuracy: 0.8232
Epoch 5/100
462/462 [==============================] - 0s 204us/step - loss: 0.3681 - accuracy: 0.8182 - val_loss: 0.3798 - val_accuracy: 0.8687
Epoch 6/100
462/462 [==============================] - 0s 310us/step - loss: 0.3498 - accuracy: 0.8658 - val_loss: 0.3725 - val_accuracy: 0.8788
Epoch 7/100
462/462 [==============================] - 0s 347us/step - loss: 0.3569 

Epoch 57/100
462/462 [==============================] - 0s 198us/step - loss: 0.0295 - accuracy: 0.9913 - val_loss: 0.0742 - val_accuracy: 0.9899
Epoch 58/100
462/462 [==============================] - 0s 237us/step - loss: 0.0265 - accuracy: 0.9957 - val_loss: 0.0776 - val_accuracy: 0.9747
Epoch 59/100
462/462 [==============================] - 0s 200us/step - loss: 0.0230 - accuracy: 0.9978 - val_loss: 0.0814 - val_accuracy: 0.9697
Epoch 60/100
462/462 [==============================] - 0s 176us/step - loss: 0.0216 - accuracy: 0.9978 - val_loss: 0.0744 - val_accuracy: 0.9697
Epoch 61/100
462/462 [==============================] - 0s 258us/step - loss: 0.0252 - accuracy: 0.9957 - val_loss: 0.0682 - val_accuracy: 0.9798
Epoch 62/100
462/462 [==============================] - 0s 232us/step - loss: 0.0233 - accuracy: 0.9957 - val_loss: 0.0701 - val_accuracy: 0.9798
Epoch 63/100
462/462 [==============================] - 0s 284us/step - loss: 0.0200 - accuracy: 0.9978 - val_loss: 0.0623 -

In [59]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

198/198 [==============================] - 0s 162us/step
over-sampling test accuracy: 96.46%


In [110]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([1, 1, 1, 0, 1, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 2, 1, 0, 2, 1,
       1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 0, 2, 0, 0, 1, 1, 1,
       0, 1, 1, 2, 2, 0, 0, 2, 1, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 1, 2, 2,
       0, 0, 1, 2, 2, 0, 0, 1, 2, 0, 1, 2, 1, 1, 0, 1, 2, 1, 2, 0, 0, 1,
       0, 0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 2, 0, 0, 1, 2, 2, 1, 1, 2, 0,
       1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 0, 0, 2, 1, 1, 0, 1, 2, 2, 2, 0, 1,
       0, 2, 1, 2, 0, 0, 1, 2, 1, 2, 2, 0, 0, 2, 0, 1, 1, 0, 0, 0, 1, 2,
       2, 2, 1, 1, 1, 2, 2, 0, 1, 2, 1, 0, 1, 2, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 2, 1, 1, 1, 0, 2, 0, 0, 0, 0, 2, 0, 2, 2, 1, 0, 2, 0, 2])

In [111]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,NRS110,1,1
1,NRS216,1,1
2,NRS386,1,1
3,CFBRSa25,0,0
4,BCH-SA-03,1,1
...,...,...,...
193,NRS216,1,1
194,CFBREBSa110,0,0
195,NRS148,2,2
196,GA27,0,0


In [112]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [113]:
dat_proba4

,0,1,2
0,2.220611e-03,0.997779,1.605854e-09
1,6.468139e-03,0.993532,1.171246e-07
2,2.556950e-02,0.974430,4.912491e-07
3,9.998289e-01,0.000171,1.743965e-10
4,6.377019e-03,0.993623,6.990759e-08
...,...,...,...
193,6.468146e-03,0.993532,1.171247e-07
194,9.994338e-01,0.000566,2.796447e-10
195,1.941034e-07,0.000874,9.991261e-01
196,9.970651e-01,0.002935,9.256775e-14


In [114]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [115]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p002yp.csv", index = False,
         header=None)

In [63]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 270us/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 0.1158 - val_accuracy: 0.9646
Epoch 2/100
462/462 [==============================] - 0s 229us/step - loss: 0.0099 - accuracy: 1.0000 - val_loss: 0.1190 - val_accuracy: 0.9646
Epoch 3/100
462/462 [==============================] - 0s 319us/step - loss: 0.0069 - accuracy: 1.0000 - val_loss: 0.1006 - val_accuracy: 0.9646
Epoch 4/100
462/462 [==============================] - 0s 317us/step - loss: 0.0059 - accuracy: 1.0000 - val_loss: 0.0972 - val_accuracy: 0.9747
Epoch 5/100
462/462 [==============================] - 0s 299us/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.1108 - val_accuracy: 0.9697
Epoch 6/100
462/462 [==============================] - 0s 300us/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.1319 - val_accuracy: 0.9646
Epoch 7/100
462/462 [==============================] - 0s 259us/step - loss: 0.0045 

In [64]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 100.00%


In [135]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [136]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [137]:
y_prob4 = df_proba4[df_proba4['phage']=='p002ypresabs_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[2.22061090e-03, 9.97779400e-01, 1.60585430e-09],
       [6.46813900e-03, 9.93531760e-01, 1.17124640e-07],
       [2.55695020e-02, 9.74430000e-01, 4.91249070e-07],
       [9.99828930e-01, 1.71014570e-04, 1.74396480e-10],
       [6.37701900e-03, 9.93622840e-01, 6.99075850e-08],
       [2.22061090e-03, 9.97779400e-01, 1.60585430e-09],
       [2.61737020e-05, 2.23427410e-05, 9.99951500e-01],
       [2.61737020e-05, 2.23427410e-05, 9.99951500e-01],
       [2.61737020e-05, 2.23427410e-05, 9.99951500e-01],
       [9.99816000e-01, 1.81194740e-04, 2.90974530e-06],
       [1.94103050e-07, 8.73678360e-04, 9.99126140e-01],
       [9.88741800e-01, 1.12580920e-02, 1.38097720e-07],
       [9.97488600e-01, 2.51024980e-03, 1.09917700e-06],
       [9.89710750e-01, 1.02892240e-02, 7.03940060e-09],
       [9.99883060e-01, 1.16935730e-04, 1.54633320e-10],
       [2.61737020e-05, 2.23427410e-05, 9.99951500e-01],
       [9.99993900e-01, 6.09301300e-06, 2.94204860e-09],
       [2.61737020e-05, 2.23427

In [138]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.9970156106519742

In [139]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.9970156106519742

In [140]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.9964512549739822

In [141]:
np.std(ovos)

0.0017448573181583576

In [142]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.9964512549739822

In [143]:
np.std(ovrs)

0.0017448573181583576

In [65]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [66]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 97.60%


In [67]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.009698407745857654


In [68]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [69]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 100.00%


In [70]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 3.201843e-05


In [144]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_over)

In [145]:
dat5 = pd.DataFrame(X_test_over[:,0])
dat5['test'] = y_test_over

In [146]:
dat5

,0,test
0,NRS241,1
1,NRS148,2
2,NRS255,1
3,NRS214,0
4,NRS148,2
...,...,...
193,CFBRSa30,0
194,NRS266,1
195,SR4152,0
196,NRS109,2


In [147]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [148]:
#### add regularizer and dropout
model1_over5 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [121]:
model1_over5.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [122]:
model1_over5.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 1ms/step - loss: 4.1172 - accuracy: 0.4545 - val_loss: 0.9910 - val_accuracy: 0.6970
Epoch 2/100
462/462 [==============================] - 0s 336us/step - loss: 3.3673 - accuracy: 0.5801 - val_loss: 0.9944 - val_accuracy: 0.7172
Epoch 3/100
462/462 [==============================] - 0s 316us/step - loss: 3.4016 - accuracy: 0.6061 - val_loss: 0.8445 - val_accuracy: 0.7071
Epoch 4/100
462/462 [==============================] - 0s 235us/step - loss: 3.6975 - accuracy: 0.5801 - val_loss: 0.8747 - val_accuracy: 0.7121
Epoch 5/100
462/462 [==============================] - 0s 234us/step - loss: 3.1697 - accuracy: 0.6061 - val_loss: 0.8867 - val_accuracy: 0.6717
Epoch 6/100
462/462 [==============================] - 0s 414us/step - loss: 3.3122 - accuracy: 0.5996 - val_loss: 0.9408 - val_accuracy: 0.6667
Epoch 7/100
462/462 [==============================] - 0s 332us/step - loss: 3.3610 - 

Epoch 57/100
462/462 [==============================] - 0s 202us/step - loss: 0.9570 - accuracy: 0.8182 - val_loss: 0.6222 - val_accuracy: 0.9545
Epoch 58/100
462/462 [==============================] - 0s 235us/step - loss: 0.9745 - accuracy: 0.7792 - val_loss: 0.8407 - val_accuracy: 0.9343
Epoch 59/100
462/462 [==============================] - 0s 184us/step - loss: 0.9554 - accuracy: 0.7597 - val_loss: 0.7853 - val_accuracy: 0.9444
Epoch 60/100
462/462 [==============================] - 0s 200us/step - loss: 1.0205 - accuracy: 0.7771 - val_loss: 0.6290 - val_accuracy: 0.9545
Epoch 61/100
462/462 [==============================] - 0s 249us/step - loss: 1.0046 - accuracy: 0.7792 - val_loss: 0.8076 - val_accuracy: 0.8838
Epoch 62/100
462/462 [==============================] - 0s 210us/step - loss: 0.9394 - accuracy: 0.7792 - val_loss: 0.7223 - val_accuracy: 0.9495
Epoch 63/100
462/462 [==============================] - 0s 211us/step - loss: 0.9490 - accuracy: 0.7749 - val_loss: 0.6824 -

In [104]:
acc_test_over5 = model1_over5.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over5*100))

198/198 [==============================] - 0s 268us/step
over-sampling test accuracy: 96.46%


In [123]:
pred5 = model1_over5.predict_classes(X_test_over)
pred5

array([1, 2, 1, 0, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 0, 2, 2, 1, 2, 2, 1, 1,
       2, 2, 1, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 1, 1, 2,
       0, 0, 1, 2, 2, 0, 0, 1, 1, 0, 1, 0, 2, 0, 1, 1, 2, 2, 2, 2, 0, 2,
       2, 2, 0, 1, 2, 1, 0, 0, 0, 2, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 0, 1,
       2, 1, 0, 1, 2, 0, 2, 1, 1, 0, 2, 0, 2, 0, 1, 2, 0, 1, 1, 1, 2, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 2, 2, 0, 1, 0, 2, 1,
       1, 1, 1, 0, 1, 0, 2, 0, 0, 1, 1, 2, 0, 1, 0, 2, 1, 1, 1, 1, 0, 2,
       0, 1, 0, 0, 2, 1, 1, 1, 1, 2, 2, 1, 2, 0, 1, 0, 2, 1, 2, 1, 1, 2,
       2, 1, 2, 2, 0, 2, 0, 2, 2, 2, 1, 1, 0, 2, 1, 0, 1, 0, 1, 0, 2, 1])

In [124]:
dat5['pred'] = pred5
dat5

,0,test,pred
0,NRS241,1,1
1,NRS148,2,2
2,NRS255,1,1
3,NRS214,0,0
4,NRS148,2,2
...,...,...,...
193,CFBRSa30,0,0
194,NRS266,1,1
195,SR4152,0,0
196,NRS109,2,2


In [125]:
proba5 = model1_over5.predict_proba(X_test_over)
dat_proba5 = pd.DataFrame(proba5)

In [126]:
dat_proba5

,0,1,2
0,1.643635e-04,9.998355e-01,1.573355e-07
1,1.189490e-08,7.780775e-09,1.000000e+00
2,4.533502e-09,1.000000e+00,2.759057e-11
3,1.000000e+00,2.952921e-10,1.368696e-10
4,1.189490e-08,7.780775e-09,1.000000e+00
...,...,...,...
193,1.000000e+00,1.333206e-14,7.526777e-15
194,2.031151e-08,1.000000e+00,3.025762e-10
195,1.000000e+00,1.780141e-11,3.428157e-12
196,9.000139e-09,5.614254e-09,1.000000e+00


In [127]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [128]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p002yp.csv", index = False,
         header=None)

In [108]:
hist1_over5 = model1_over5.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 345us/step - loss: 0.4620 - accuracy: 0.7857 - val_loss: 0.5468 - val_accuracy: 0.9697
Epoch 2/100
462/462 [==============================] - 0s 286us/step - loss: 0.4935 - accuracy: 0.7792 - val_loss: 0.6729 - val_accuracy: 0.9596
Epoch 3/100
462/462 [==============================] - 0s 255us/step - loss: 0.5128 - accuracy: 0.8052 - val_loss: 0.6165 - val_accuracy: 0.9596
Epoch 4/100
462/462 [==============================] - 0s 233us/step - loss: 0.4904 - accuracy: 0.7987 - val_loss: 0.6004 - val_accuracy: 0.9596
Epoch 5/100
462/462 [==============================] - 0s 248us/step - loss: 0.6076 - accuracy: 0.7597 - val_loss: 0.6550 - val_accuracy: 0.9646
Epoch 6/100
462/462 [==============================] - 0s 233us/step - loss: 0.4876 - accuracy: 0.7857 - val_loss: 0.6224 - val_accuracy: 0.9646
Epoch 7/100
462/462 [==============================] - 0s 225us/step - loss: 0.4549 

Epoch 57/100
462/462 [==============================] - 0s 217us/step - loss: 0.5319 - accuracy: 0.7965 - val_loss: 0.6154 - val_accuracy: 0.9646
Epoch 58/100
462/462 [==============================] - 0s 207us/step - loss: 0.5877 - accuracy: 0.8160 - val_loss: 0.9649 - val_accuracy: 0.9394
Epoch 59/100
462/462 [==============================] - 0s 210us/step - loss: 0.5307 - accuracy: 0.7965 - val_loss: 0.6239 - val_accuracy: 0.9545
Epoch 60/100
462/462 [==============================] - 0s 209us/step - loss: 0.4545 - accuracy: 0.7879 - val_loss: 0.4494 - val_accuracy: 0.9697
Epoch 61/100
462/462 [==============================] - 0s 209us/step - loss: 0.4582 - accuracy: 0.7900 - val_loss: 0.6009 - val_accuracy: 0.9646
Epoch 62/100
462/462 [==============================] - 0s 210us/step - loss: 0.4126 - accuracy: 0.7965 - val_loss: 0.5013 - val_accuracy: 0.9646
Epoch 63/100
462/462 [==============================] - 0s 214us/step - loss: 0.3210 - accuracy: 0.8355 - val_loss: 0.4409 -

In [109]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over5.history['accuracy'])*100))

over-sampling train accuracy: 79.68%


In [149]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [150]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS241,1,1,1.342914e-03,9.986569e-01,2.348628e-07
1,p002ykpresabsSTCC_qual,NRS148,2,2,5.170289e-08,1.017893e-07,9.999999e-01
2,p002ykpresabsSTCC_qual,NRS255,1,1,1.780311e-07,9.999999e-01,2.544841e-12
3,p002ykpresabsSTCC_qual,NRS214,0,0,1.000000e+00,2.203547e-10,5.688883e-15
4,p002ykpresabsSTCC_qual,NRS148,2,2,5.170289e-08,1.017893e-07,9.999999e-01
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,BCH-SA-12,0,0,1.000000e+00,1.152503e-09,1.898730e-09
1978,pyopresabsSTCC_qual,NRS049,0,1,8.401357e-11,1.000000e+00,3.209735e-13
1979,pyopresabsSTCC_qual,NRS022,0,0,1.000000e+00,4.755084e-10,1.974275e-10
1980,pyopresabsSTCC_qual,NRS236,1,1,1.357345e-08,1.000000e+00,1.293117e-10


In [151]:
y_prob5 = df_proba5[df_proba5['phage']=='p002ypresabs_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[1.64363530e-04, 9.99835500e-01, 1.57335550e-07],
       [1.18948990e-08, 7.78077500e-09, 1.00000000e+00],
       [4.53350250e-09, 1.00000000e+00, 2.75905690e-11],
       [1.00000000e+00, 2.95292100e-10, 1.36869630e-10],
       [1.18948990e-08, 7.78077500e-09, 1.00000000e+00],
       [8.61057800e-09, 1.00000000e+00, 3.41442740e-12],
       [9.00012200e-09, 5.61425350e-09, 1.00000000e+00],
       [3.85139740e-08, 3.12501850e-08, 9.99999900e-01],
       [1.12303155e-07, 9.99999900e-01, 4.11950300e-09],
       [2.32923520e-05, 9.99976750e-01, 3.82343800e-11],
       [1.18948990e-08, 7.78077500e-09, 1.00000000e+00],
       [3.85139740e-08, 3.12501850e-08, 9.99999900e-01],
       [9.00012200e-09, 5.61425350e-09, 1.00000000e+00],
       [1.78272530e-07, 9.99999900e-01, 8.30041500e-09],
       [1.00000000e+00, 1.82837600e-22, 1.54079860e-22],
       [9.00012200e-09, 5.61425350e-09, 1.00000000e+00],
       [1.18948990e-08, 7.78077500e-09, 1.00000000e+00],
       [2.35751900e-07, 9.99999

In [152]:
ovo5 = rocauc_ovo(y_test_over, y_prob5, average="macro", multi_class="ovo")
ovo5

0.9908363942454853

In [153]:
ovr5 = rocauc_ovr(y_test_over, y_prob5, average="macro", multi_class="ovr")
ovr5

0.9908363942454853

In [154]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_over)

In [155]:
dat6 = pd.DataFrame(X_test_over[:,0])
dat6['test'] = y_test_over

In [156]:
dat6

,0,test
0,NRS209,2
1,NRS386,1
2,NRS148,2
3,NRS178,0
4,NRS237,0
...,...,...
193,NRS209,2
194,NRS002,0
195,NRS109,2
196,BCH-SA-03,1


In [157]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [133]:
model1_over6 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [134]:
model1_over6.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [135]:
model1_over6.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 828us/step - loss: 3.7536 - accuracy: 0.3918 - val_loss: 0.9904 - val_accuracy: 0.5152
Epoch 2/100
462/462 [==============================] - 0s 225us/step - loss: 3.6162 - accuracy: 0.5281 - val_loss: 0.9305 - val_accuracy: 0.6869
Epoch 3/100
462/462 [==============================] - 0s 258us/step - loss: 3.8724 - accuracy: 0.5974 - val_loss: 0.9222 - val_accuracy: 0.6919
Epoch 4/100
462/462 [==============================] - 0s 373us/step - loss: 3.8182 - accuracy: 0.6082 - val_loss: 0.8911 - val_accuracy: 0.6768
Epoch 5/100
462/462 [==============================] - 0s 343us/step - loss: 3.2671 - accuracy: 0.6255 - val_loss: 0.8770 - val_accuracy: 0.7323
Epoch 6/100
462/462 [==============================] - 0s 197us/step - loss: 3.4186 - accuracy: 0.6558 - val_loss: 0.7635 - val_accuracy: 0.7222
Epoch 7/100
462/462 [==============================] - 0s 188us/step - loss: 3.5538 

Epoch 57/100
462/462 [==============================] - 0s 218us/step - loss: 0.7730 - accuracy: 0.7706 - val_loss: 0.4431 - val_accuracy: 0.9293
Epoch 58/100
462/462 [==============================] - 0s 205us/step - loss: 0.8811 - accuracy: 0.7597 - val_loss: 0.7746 - val_accuracy: 0.9444
Epoch 59/100
462/462 [==============================] - 0s 338us/step - loss: 0.8798 - accuracy: 0.7489 - val_loss: 0.5748 - val_accuracy: 0.9444
Epoch 60/100
462/462 [==============================] - 0s 305us/step - loss: 0.8909 - accuracy: 0.7792 - val_loss: 0.6253 - val_accuracy: 0.9495
Epoch 61/100
462/462 [==============================] - 0s 235us/step - loss: 0.7932 - accuracy: 0.7879 - val_loss: 0.8435 - val_accuracy: 0.9394
Epoch 62/100
462/462 [==============================] - 0s 419us/step - loss: 1.2250 - accuracy: 0.7662 - val_loss: 0.8024 - val_accuracy: 0.8687
Epoch 63/100
462/462 [==============================] - 0s 321us/step - loss: 1.0406 - accuracy: 0.7554 - val_loss: 0.4022 -

In [117]:
acc_test_over6 = model1_over6.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over6*100))

198/198 [==============================] - 0s 171us/step
over-sampling test accuracy: 92.42%


In [136]:
pred6 = model1_over6.predict_classes(X_test_over)
pred6

array([2, 1, 2, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 2, 0,
       0, 0, 1, 2, 1, 2, 2, 0, 1, 0, 2, 2, 1, 0, 0, 2, 0, 2, 1, 2, 1, 2,
       2, 2, 2, 0, 0, 1, 2, 0, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 0, 1, 2, 2,
       0, 0, 1, 2, 1, 0, 1, 2, 0, 1, 0, 2, 1, 2, 2, 1, 0, 2, 0, 1, 0, 0,
       0, 2, 2, 0, 1, 0, 2, 0, 0, 1, 2, 1, 0, 1, 2, 0, 1, 2, 2, 1, 0, 0,
       2, 1, 0, 2, 1, 1, 2, 1, 2, 1, 1, 1, 0, 2, 2, 1, 2, 0, 1, 0, 1, 0,
       1, 1, 2, 0, 2, 2, 0, 1, 0, 1, 1, 0, 2, 0, 2, 1, 0, 2, 0, 2, 1, 2,
       1, 2, 1, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1,
       0, 1, 1, 0, 0, 2, 1, 0, 1, 1, 1, 0, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1])

In [137]:
dat6['pred'] = pred6
dat6

,0,test,pred
0,NRS209,2,2
1,NRS386,1,1
2,NRS148,2,2
3,NRS178,0,1
4,NRS237,0,0
...,...,...,...
193,NRS209,2,2
194,NRS002,0,1
195,NRS109,2,2
196,BCH-SA-03,1,1


In [138]:
proba6 = model1_over6.predict_proba(X_test_over)
dat_proba6 = pd.DataFrame(proba6)

In [139]:
dat_proba6

,0,1,2
0,1.033055e-08,2.077004e-08,1.000000e+00
1,2.341317e-02,9.765866e-01,1.195465e-07
2,1.534076e-09,8.725181e-09,1.000000e+00
3,4.741061e-19,1.000000e+00,1.490597e-17
4,1.000000e+00,2.908509e-11,5.167130e-14
...,...,...,...
193,1.033055e-08,2.077004e-08,1.000000e+00
194,1.456927e-02,9.854306e-01,9.453345e-08
195,1.542550e-09,1.027252e-08,1.000000e+00
196,9.916769e-08,9.999999e-01,7.587334e-10


In [140]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [141]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p002yp.csv", index = False,
         header=None)

In [121]:
hist1_over6 = model1_over6.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 483us/step - loss: 0.5355 - accuracy: 0.8268 - val_loss: 0.6524 - val_accuracy: 0.9495
Epoch 2/100
462/462 [==============================] - 0s 445us/step - loss: 0.4881 - accuracy: 0.8268 - val_loss: 0.6525 - val_accuracy: 0.9495
Epoch 3/100
462/462 [==============================] - 0s 283us/step - loss: 0.5805 - accuracy: 0.7792 - val_loss: 0.7863 - val_accuracy: 0.9495
Epoch 4/100
462/462 [==============================] - 0s 264us/step - loss: 0.5213 - accuracy: 0.7922 - val_loss: 0.7767 - val_accuracy: 0.9495
Epoch 5/100
462/462 [==============================] - 0s 235us/step - loss: 0.5295 - accuracy: 0.8095 - val_loss: 0.4984 - val_accuracy: 0.9495
Epoch 6/100
462/462 [==============================] - 0s 261us/step - loss: 0.4479 - accuracy: 0.8030 - val_loss: 0.5573 - val_accuracy: 0.9495
Epoch 7/100
462/462 [==============================] - 0s 227us/step - loss: 0.5694 

In [123]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over6.history['accuracy'])*100))

over-sampling train accuracy: 79.44%


In [158]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [159]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS209,2,2,1.790400e-08,4.141849e-08,1.000000e+00
1,p002ykpresabsSTCC_qual,NRS386,1,1,5.739934e-04,9.994259e-01,6.773014e-08
2,p002ykpresabsSTCC_qual,NRS148,2,2,5.286934e-09,1.269109e-08,1.000000e+00
3,p002ykpresabsSTCC_qual,NRS178,0,1,6.494936e-12,1.000000e+00,2.537080e-25
4,p002ykpresabsSTCC_qual,NRS237,0,1,5.701098e-02,9.399204e-01,3.068583e-03
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS272,0,0,9.999607e-01,3.367024e-05,5.776848e-06
1978,pyopresabsSTCC_qual,NRS112,1,1,8.275442e-08,9.999999e-01,3.739556e-09
1979,pyopresabsSTCC_qual,NRS064,1,1,2.168245e-08,1.000000e+00,9.603962e-09
1980,pyopresabsSTCC_qual,BCH-SA-04,0,0,1.000000e+00,1.026408e-15,1.630406e-14


In [160]:
y_prob6 = df_proba6[df_proba6['phage']=='p002ypresabs_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[1.03305460e-08, 2.07700360e-08, 1.00000000e+00],
       [2.34131740e-02, 9.76586640e-01, 1.19546480e-07],
       [1.53407590e-09, 8.72518150e-09, 1.00000000e+00],
       [4.74106100e-19, 1.00000000e+00, 1.49059670e-17],
       [1.00000000e+00, 2.90850850e-11, 5.16712970e-14],
       [1.53128040e-10, 1.00000000e+00, 5.85402500e-12],
       [9.91675100e-08, 9.99999900e-01, 7.58733350e-10],
       [1.03305460e-08, 2.07700360e-08, 1.00000000e+00],
       [1.53128040e-10, 1.00000000e+00, 5.85402500e-12],
       [3.27097680e-01, 6.54188330e-01, 1.87139640e-02],
       [2.34131740e-02, 9.76586640e-01, 1.19546480e-07],
       [1.94771740e-01, 8.05226300e-01, 1.90955700e-06],
       [7.69961800e-07, 9.99998450e-01, 8.56489900e-07],
       [4.04378950e-04, 9.99595600e-01, 1.09082540e-08],
       [1.00000000e+00, 1.33622100e-13, 6.24817600e-17],
       [9.91675100e-08, 9.99999900e-01, 7.58733350e-10],
       [1.00000000e+00, 2.36278970e-10, 7.05662700e-13],
       [1.02297610e-06, 9.99997

In [161]:
ovo6 = rocauc_ovo(y_test_over, y_prob6, average="macro", multi_class="ovo")
ovo6

0.9923094582185491

In [162]:
ovr6 = rocauc_ovr(y_test_over, y_prob6, average="macro", multi_class="ovr")
ovr6

0.9923094582185491

In [163]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_over)

In [164]:
dat7 = pd.DataFrame(X_test_over[:,0])
dat7['test'] = y_test_over

In [165]:
dat7

,0,test
0,NRS209,2
1,BCH-SA-09,1
2,NRS224,0
3,NRS209,2
4,NRS235,1
...,...,...
193,NRS209,2
194,CFBREBSa131,1
195,CFBREBSa103,0
196,NRS188,1


In [166]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [146]:
model1_over7 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [147]:
model1_over7.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [148]:
model1_over7.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 741us/step - loss: 3.8863 - accuracy: 0.5195 - val_loss: 0.9374 - val_accuracy: 0.7525
Epoch 2/100
462/462 [==============================] - 0s 359us/step - loss: 3.4163 - accuracy: 0.6710 - val_loss: 0.8999 - val_accuracy: 0.7626
Epoch 3/100
462/462 [==============================] - 0s 464us/step - loss: 3.2407 - accuracy: 0.6797 - val_loss: 0.8612 - val_accuracy: 0.7828
Epoch 4/100
462/462 [==============================] - 0s 242us/step - loss: 3.1458 - accuracy: 0.6710 - val_loss: 1.0645 - val_accuracy: 0.7828
Epoch 5/100
462/462 [==============================] - 0s 417us/step - loss: 2.7088 - accuracy: 0.7273 - val_loss: 0.9697 - val_accuracy: 0.6970
Epoch 6/100
462/462 [==============================] - 0s 392us/step - loss: 3.5323 - accuracy: 0.6818 - val_loss: 1.0858 - val_accuracy: 0.8131
Epoch 7/100
462/462 [==============================] - 0s 338us/step - loss: 3.2311 

Epoch 57/100
462/462 [==============================] - 0s 243us/step - loss: 0.7658 - accuracy: 0.7727 - val_loss: 0.9558 - val_accuracy: 0.9293
Epoch 58/100
462/462 [==============================] - 0s 223us/step - loss: 0.7744 - accuracy: 0.7814 - val_loss: 0.6605 - val_accuracy: 0.9343
Epoch 59/100
462/462 [==============================] - 0s 248us/step - loss: 0.7919 - accuracy: 0.7965 - val_loss: 1.1242 - val_accuracy: 0.8939
Epoch 60/100
462/462 [==============================] - 0s 253us/step - loss: 0.9563 - accuracy: 0.7684 - val_loss: 0.8282 - val_accuracy: 0.9394
Epoch 61/100
462/462 [==============================] - 0s 218us/step - loss: 0.7563 - accuracy: 0.7749 - val_loss: 0.8894 - val_accuracy: 0.9394
Epoch 62/100
462/462 [==============================] - 0s 503us/step - loss: 0.6029 - accuracy: 0.7900 - val_loss: 0.6624 - val_accuracy: 0.9394
Epoch 63/100
462/462 [==============================] - 0s 290us/step - loss: 0.7238 - accuracy: 0.7987 - val_loss: 0.8330 -

In [131]:
acc_test_over7 = model1_over7.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over7*100))

198/198 [==============================] - 0s 340us/step
over-sampling test accuracy: 94.44%


In [149]:
pred7 = model1_over7.predict_classes(X_test_over)
pred7

array([2, 1, 0, 2, 1, 1, 1, 2, 1, 1, 1, 0, 0, 0, 1, 2, 1, 2, 2, 0, 2, 1,
       2, 0, 0, 2, 1, 0, 1, 0, 1, 2, 1, 2, 2, 2, 1, 2, 0, 0, 2, 1, 1, 1,
       2, 1, 1, 2, 0, 1, 2, 2, 2, 2, 0, 0, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0,
       0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 2, 0, 2, 1, 0, 1, 1, 2, 1,
       1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 0, 2, 0, 1, 2, 1, 2, 0, 0, 0,
       2, 2, 1, 1, 0, 1, 0, 1, 2, 0, 0, 1, 2, 0, 1, 0, 1, 0, 1, 0, 1, 2,
       0, 2, 2, 0, 0, 1, 2, 1, 0, 2, 1, 2, 1, 1, 2, 1, 2, 0, 2, 2, 0, 2,
       0, 0, 2, 0, 1, 2, 0, 2, 2, 2, 2, 1, 2, 2, 1, 1, 1, 2, 1, 0, 1, 2])

In [150]:
dat7['pred'] = pred7
dat7

,0,test,pred
0,NRS209,2,2
1,BCH-SA-09,1,1
2,NRS224,0,0
3,NRS209,2,2
4,NRS235,1,1
...,...,...,...
193,NRS209,2,2
194,CFBREBSa131,1,1
195,CFBREBSa103,0,0
196,NRS188,1,1


In [151]:
proba7 = model1_over7.predict_proba(X_test_over)
dat_proba7 = pd.DataFrame(proba7)

In [152]:
dat_proba7

,0,1,2
0,3.618598e-08,3.613115e-08,9.999999e-01
1,2.141211e-08,1.000000e+00,4.190622e-09
2,1.000000e+00,3.118812e-13,2.544858e-14
3,3.618598e-08,3.613115e-08,9.999999e-01
4,1.248625e-06,9.999988e-01,1.258703e-09
...,...,...,...
193,3.618598e-08,3.613115e-08,9.999999e-01
194,3.175916e-02,9.307808e-01,3.745997e-02
195,1.000000e+00,3.680168e-13,5.076824e-14
196,5.261245e-12,1.000000e+00,4.079788e-13


In [153]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [154]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p002yp.csv", index = False,
         header=None)

In [135]:
hist1_over7 = model1_over7.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 283us/step - loss: 0.6228 - accuracy: 0.7944 - val_loss: 0.7755 - val_accuracy: 0.9394
Epoch 2/100
462/462 [==============================] - 0s 234us/step - loss: 0.5683 - accuracy: 0.8074 - val_loss: 1.1025 - val_accuracy: 0.9242
Epoch 3/100
462/462 [==============================] - 0s 223us/step - loss: 0.5511 - accuracy: 0.8117 - val_loss: 0.9267 - val_accuracy: 0.9394
Epoch 4/100
462/462 [==============================] - 0s 258us/step - loss: 0.6326 - accuracy: 0.7706 - val_loss: 0.7069 - val_accuracy: 0.9394
Epoch 5/100
462/462 [==============================] - 0s 329us/step - loss: 0.5060 - accuracy: 0.8052 - val_loss: 0.8987 - val_accuracy: 0.9343
Epoch 6/100
462/462 [==============================] - 0s 301us/step - loss: 0.5383 - accuracy: 0.8117 - val_loss: 0.7937 - val_accuracy: 0.9394
Epoch 7/100
462/462 [==============================] - 0s 263us/step - loss: 0.5536 

Epoch 57/100
462/462 [==============================] - 0s 298us/step - loss: 0.4510 - accuracy: 0.7965 - val_loss: 0.5794 - val_accuracy: 0.9646
Epoch 58/100
462/462 [==============================] - 0s 496us/step - loss: 0.4428 - accuracy: 0.8009 - val_loss: 0.8526 - val_accuracy: 0.9343
Epoch 59/100
462/462 [==============================] - 0s 380us/step - loss: 0.4729 - accuracy: 0.7792 - val_loss: 0.7002 - val_accuracy: 0.9495
Epoch 60/100
462/462 [==============================] - 0s 471us/step - loss: 0.4026 - accuracy: 0.8247 - val_loss: 0.7377 - val_accuracy: 0.9394
Epoch 61/100
462/462 [==============================] - 0s 323us/step - loss: 0.4446 - accuracy: 0.7900 - val_loss: 0.7849 - val_accuracy: 0.9343
Epoch 62/100
462/462 [==============================] - 0s 269us/step - loss: 0.4475 - accuracy: 0.8247 - val_loss: 1.1224 - val_accuracy: 0.9141
Epoch 63/100
462/462 [==============================] - 0s 251us/step - loss: 0.4427 - accuracy: 0.7835 - val_loss: 0.7286 -

In [136]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over7.history['accuracy'])*100))

over-sampling train accuracy: 79.45%


In [167]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [168]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS209,2,2,8.300497e-12,1.036520e-09,1.000000e+00
1,p002ykpresabsSTCC_qual,BCH-SA-09,1,1,1.137139e-06,9.999988e-01,2.067601e-09
2,p002ykpresabsSTCC_qual,NRS224,0,0,1.000000e+00,2.093110e-31,0.000000e+00
3,p002ykpresabsSTCC_qual,NRS209,2,2,8.300497e-12,1.036520e-09,1.000000e+00
4,p002ykpresabsSTCC_qual,NRS235,1,1,2.243513e-02,9.774035e-01,1.615106e-04
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS035,0,0,9.354528e-01,6.414209e-02,4.051121e-04
1978,pyopresabsSTCC_qual,NRS260,1,1,4.808470e-08,1.000000e+00,7.364639e-09
1979,pyopresabsSTCC_qual,CA9,0,0,1.000000e+00,2.361323e-08,2.871247e-08
1980,pyopresabsSTCC_qual,NRS183,1,1,2.755864e-07,9.999998e-01,5.310879e-08


In [169]:
y_prob7 = df_proba7[df_proba7['phage']=='p002ypresabs_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[3.61859800e-08, 3.61311500e-08, 9.99999900e-01],
       [2.14121060e-08, 1.00000000e+00, 4.19062160e-09],
       [1.00000000e+00, 3.11881220e-13, 2.54485810e-14],
       [3.61859800e-08, 3.61311500e-08, 9.99999900e-01],
       [1.24862460e-06, 9.99998800e-01, 1.25870260e-09],
       [5.89275330e-08, 1.00000000e+00, 1.16629400e-08],
       [7.93851700e-10, 1.00000000e+00, 4.46804200e-11],
       [3.61859800e-08, 3.61311500e-08, 9.99999900e-01],
       [3.17591580e-02, 9.30780800e-01, 3.74599730e-02],
       [1.24862460e-06, 9.99998800e-01, 1.25870260e-09],
       [4.61083830e-08, 1.00000000e+00, 9.74188900e-09],
       [1.00000000e+00, 7.10465000e-17, 3.45499450e-18],
       [1.00000000e+00, 1.83221940e-11, 2.10740700e-12],
       [1.00000000e+00, 3.96622180e-10, 5.42163470e-11],
       [7.93851700e-10, 1.00000000e+00, 4.46804200e-11],
       [3.07662100e-09, 3.88725000e-08, 1.00000000e+00],
       [3.17591580e-02, 9.30780800e-01, 3.74599730e-02],
       [3.61859800e-08, 3.61311

In [170]:
ovo7 = rocauc_ovo(y_test_over, y_prob7, average="macro", multi_class="ovo")
ovo7

0.9906450872359963

In [171]:
ovr7 = rocauc_ovr(y_test_over, y_prob7, average="macro", multi_class="ovr")
ovr7

0.9906450872359963

In [172]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_over)

In [173]:
dat8 = pd.DataFrame(X_test_over[:,0])
dat8['test'] = y_test_over

In [174]:
dat8

,0,test
0,CFBREBSa116,0
1,NRS214,0
2,NRS148,2
3,NRS148,2
4,NRS148,2
...,...,...
193,NRS148,2
194,NRS054,0
195,NRS109,2
196,NRS216,1


In [175]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [159]:
model1_over8 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [160]:
model1_over8.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [161]:
model1_over8.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 841us/step - loss: 4.0770 - accuracy: 0.5779 - val_loss: 0.9237 - val_accuracy: 0.7020
Epoch 2/100
462/462 [==============================] - 0s 234us/step - loss: 3.4172 - accuracy: 0.6602 - val_loss: 0.8772 - val_accuracy: 0.7323
Epoch 3/100
462/462 [==============================] - 0s 404us/step - loss: 3.1248 - accuracy: 0.6775 - val_loss: 0.8833 - val_accuracy: 0.7576
Epoch 4/100
462/462 [==============================] - 0s 396us/step - loss: 3.3125 - accuracy: 0.6472 - val_loss: 1.2084 - val_accuracy: 0.7576
Epoch 5/100
462/462 [==============================] - 0s 255us/step - loss: 3.0191 - accuracy: 0.6732 - val_loss: 1.1903 - val_accuracy: 0.7626
Epoch 6/100
462/462 [==============================] - 0s 201us/step - loss: 2.7613 - accuracy: 0.6948 - val_loss: 1.9376 - val_accuracy: 0.7778
Epoch 7/100
462/462 [==============================] - 0s 665us/step - loss: 2.8846 

Epoch 57/100
462/462 [==============================] - 0s 381us/step - loss: 0.6941 - accuracy: 0.7944 - val_loss: 0.6686 - val_accuracy: 0.9293
Epoch 58/100
462/462 [==============================] - 0s 422us/step - loss: 0.8144 - accuracy: 0.7554 - val_loss: 0.4744 - val_accuracy: 0.9444
Epoch 59/100
462/462 [==============================] - 0s 267us/step - loss: 0.6599 - accuracy: 0.8182 - val_loss: 0.5971 - val_accuracy: 0.9444
Epoch 60/100
462/462 [==============================] - 0s 212us/step - loss: 0.6926 - accuracy: 0.8074 - val_loss: 0.5227 - val_accuracy: 0.9394
Epoch 61/100
462/462 [==============================] - 0s 229us/step - loss: 0.7958 - accuracy: 0.7792 - val_loss: 0.5680 - val_accuracy: 0.9444
Epoch 62/100
462/462 [==============================] - 0s 571us/step - loss: 0.5803 - accuracy: 0.7987 - val_loss: 0.4787 - val_accuracy: 0.9394
Epoch 63/100
462/462 [==============================] - 0s 278us/step - loss: 0.7285 - accuracy: 0.7771 - val_loss: 0.8289 -

In [144]:
acc_test_over8 = model1_over8.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over8*100))

198/198 [==============================] - 0s 164us/step
over-sampling test accuracy: 95.96%


In [162]:
pred8 = model1_over8.predict_classes(X_test_over)
pred8

array([0, 0, 2, 2, 2, 1, 2, 0, 0, 1, 0, 2, 1, 0, 1, 2, 1, 2, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 2, 1, 0, 2, 1, 0, 1, 1, 2, 2, 0, 0, 0, 2, 0, 1, 2,
       2, 2, 2, 1, 1, 1, 2, 2, 1, 0, 2, 1, 1, 2, 1, 0, 0, 1, 2, 0, 2, 2,
       1, 2, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 2, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 2, 0, 2, 0, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 0,
       2, 1, 1, 2, 2, 2, 0, 1, 1, 2, 2, 1, 0, 2, 2, 1, 1, 2, 2, 2, 2, 0,
       0, 2, 2, 1, 1, 0, 1, 2, 1, 0, 0, 1, 2, 0, 0, 2, 1, 0, 0, 0, 2, 2,
       2, 2, 2, 1, 1, 1, 1, 2, 0, 0, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2,
       2, 0, 2, 1, 0, 0, 1, 2, 1, 0, 0, 1, 2, 0, 1, 2, 0, 2, 1, 2, 1, 1])

In [163]:
dat8['pred'] = pred8
dat8

,0,test,pred
0,CFBREBSa116,0,0
1,NRS214,0,0
2,NRS148,2,2
3,NRS148,2,2
4,NRS148,2,2
...,...,...,...
193,NRS148,2,2
194,NRS054,0,1
195,NRS109,2,2
196,NRS216,1,1


In [164]:
proba8 = model1_over8.predict_proba(X_test_over)
dat_proba8 = pd.DataFrame(proba8)

In [165]:
dat_proba8

,0,1,2
0,1.000000e+00,4.325985e-11,6.464015e-12
1,9.999999e-01,1.727631e-07,2.922883e-08
2,1.915769e-09,1.829717e-09,1.000000e+00
3,1.915769e-09,1.829717e-09,1.000000e+00
4,1.915769e-09,1.829717e-09,1.000000e+00
...,...,...,...
193,1.915772e-09,1.829721e-09,1.000000e+00
194,4.625608e-01,5.088866e-01,2.855253e-02
195,6.087012e-09,6.090844e-09,1.000000e+00
196,7.759023e-12,1.000000e+00,1.633419e-11


In [166]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [167]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p002yp.csv", index = False,
         header=None)

In [148]:
hist1_over8 = model1_over8.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 313us/step - loss: 0.5796 - accuracy: 0.7771 - val_loss: 0.4017 - val_accuracy: 0.9596
Epoch 2/100
462/462 [==============================] - 0s 266us/step - loss: 0.6698 - accuracy: 0.7987 - val_loss: 0.8864 - val_accuracy: 0.9242
Epoch 3/100
462/462 [==============================] - 0s 245us/step - loss: 0.6863 - accuracy: 0.7835 - val_loss: 0.5297 - val_accuracy: 0.9242
Epoch 4/100
462/462 [==============================] - 0s 241us/step - loss: 0.5979 - accuracy: 0.7857 - val_loss: 0.7192 - val_accuracy: 0.9394
Epoch 5/100
462/462 [==============================] - 0s 269us/step - loss: 0.5544 - accuracy: 0.7965 - val_loss: 0.8611 - val_accuracy: 0.9343
Epoch 6/100
462/462 [==============================] - 0s 306us/step - loss: 0.6128 - accuracy: 0.7987 - val_loss: 0.4887 - val_accuracy: 0.9343
Epoch 7/100
462/462 [==============================] - 0s 324us/step - loss: 0.8326 

Epoch 57/100
462/462 [==============================] - 0s 250us/step - loss: 0.4498 - accuracy: 0.8182 - val_loss: 0.4518 - val_accuracy: 0.9596
Epoch 58/100
462/462 [==============================] - 0s 331us/step - loss: 0.4961 - accuracy: 0.7792 - val_loss: 0.7045 - val_accuracy: 0.9495
Epoch 59/100
462/462 [==============================] - 0s 501us/step - loss: 0.4649 - accuracy: 0.7965 - val_loss: 0.5898 - val_accuracy: 0.9545
Epoch 60/100
462/462 [==============================] - 0s 321us/step - loss: 0.5044 - accuracy: 0.7662 - val_loss: 0.7685 - val_accuracy: 0.9444
Epoch 61/100
462/462 [==============================] - 0s 203us/step - loss: 0.4076 - accuracy: 0.8074 - val_loss: 0.5753 - val_accuracy: 0.9545
Epoch 62/100
462/462 [==============================] - 0s 274us/step - loss: 0.4119 - accuracy: 0.8052 - val_loss: 0.3661 - val_accuracy: 0.9697
Epoch 63/100
462/462 [==============================] - 0s 453us/step - loss: 0.4403 - accuracy: 0.8117 - val_loss: 1.0949 -

In [149]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over8.history['accuracy'])*100))

over-sampling train accuracy: 79.51%


In [176]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [177]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,CFBREBSa116,0,0,9.676203e-01,3.237956e-02,1.480166e-07
1,p002ykpresabsSTCC_qual,NRS214,0,0,1.000000e+00,6.534852e-11,2.250731e-18
2,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
3,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
4,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS205,2,2,3.691095e-08,3.571927e-08,9.999999e-01
1978,pyopresabsSTCC_qual,CFBREBSa122,0,1,9.261665e-02,9.073822e-01,1.162373e-06
1979,pyopresabsSTCC_qual,NRS001,1,1,4.174278e-07,9.999995e-01,3.254024e-09
1980,pyopresabsSTCC_qual,NRS148,2,2,3.234670e-08,3.121212e-08,9.999999e-01


In [178]:
y_prob8 = df_proba8[df_proba8['phage']=='p002ypresabs_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[1.00000000e+00, 4.32598470e-11, 6.46401470e-12],
       [9.99999900e-01, 1.72763110e-07, 2.92288300e-08],
       [1.91576890e-09, 1.82971730e-09, 1.00000000e+00],
       [1.91576890e-09, 1.82971730e-09, 1.00000000e+00],
       [1.91576890e-09, 1.82971730e-09, 1.00000000e+00],
       [7.75903700e-12, 1.00000000e+00, 1.63342480e-11],
       [1.91576890e-09, 1.82971730e-09, 1.00000000e+00],
       [1.00000000e+00, 2.17938640e-11, 3.22324570e-12],
       [9.99999900e-01, 1.26572430e-07, 5.35242870e-09],
       [8.39436960e-08, 9.99999900e-01, 2.54192580e-08],
       [1.00000000e+00, 3.93525350e-09, 6.29118900e-10],
       [6.08697700e-09, 6.09082100e-09, 1.00000000e+00],
       [8.39436960e-08, 9.99999900e-01, 2.54192580e-08],
       [9.99998200e-01, 1.51068930e-06, 2.62199700e-07],
       [3.28353660e-11, 1.00000000e+00, 3.15474080e-12],
       [5.40744030e-09, 5.52454570e-09, 1.00000000e+00],
       [9.93080100e-02, 8.49688900e-01, 5.10030950e-02],
       [1.91576890e-09, 1.82971

In [179]:
ovo8 = rocauc_ovo(y_test_over, y_prob8, average="macro", multi_class="ovo")
ovo8

0.9788988368533823

In [180]:
ovr8 = rocauc_ovr(y_test_over, y_prob8, average="macro", multi_class="ovr")
ovr8

0.9788988368533823

In [181]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.9881724441383533

In [182]:
np.std(ovos2)

0.005392709700076941

In [183]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.9881724441383533

In [184]:
np.std(ovrs2)

0.005392709700076941

In [150]:
accs_reg = [acc_test_over5, acc_test_over6, acc_test_over7, acc_test_over8]

In [151]:
mean_reg = np.mean(accs_reg)
print('over-sampling test accuracy regularization mean: %.2f%%' % (mean_reg*100))

over-sampling test accuracy regularization mean: 94.82%


In [152]:
std_reg = np.std(accs_reg)
print('over-sampling test accuracy regularization standard deviation:', std_reg)

over-sampling test accuracy regularization standard deviation: 0.01571956877289432


In [153]:
accs_train_reg = [np.mean(hist1_over5.history['accuracy']), np.mean(hist1_over6.history['accuracy']), np.mean(hist1_over7.history['accuracy']),
             np.mean(hist1_over8.history['accuracy'])]

In [154]:
mean_train_reg = np.mean(accs_train_reg)
print('over-sampling train accuracy regularization mean: %.2f%%' % (mean_train_reg*100))

over-sampling train accuracy regularization mean: 79.52%


In [155]:
std_train_reg = np.std(accs_train_reg)
print('over-sampling train accuracy regularization standard deviation:', std_train_reg)

over-sampling train accuracy regularization standard deviation: 0.00096449774
